# Top Repositories For Github Topics

Project outline

- scrape URL: https://github.com/topics
- topic title, URL and Description
- top 20 repositpories from each topics
- grab repo name,username, stars and URL


## Pick a site and describe

1. browse
2. Identify info
3. Summarize

## Request lib to download

1. identify the URL
2. save Page
3. create func for automate downlonding

In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
topics_url = 'https://github.com/topics'

In [ ]:
resp = requests.get(topics_url)

In [ ]:
len(resp.text)

126663

In [ ]:
page_content = resp.text
page_content[:1000]

'\n\n<!DOCTYPE html>\n<html lang="en" >\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-7KjiGvJiLLy6LJPGf3m67ejAdgQsgDdnxZYoaI6+Agd0ZxHKTCjoKZgaf3PgUjURCcVceAwySJJJWgitRskDiA==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-eca8e21af2622cbcba2c93c67f79baed.css" />\n    <link crossorigin="anonymous" media="all" integrity="sha512-8b7omPlP/yEmgGwzty07UCWwxjJLdNp3WAiXPEnS7mZvEvNKdiuLXT681i7qJujik30dRviXyNkKAz9F4w94UA==" rel="stylesheet" href="https://github.githubassets.com/assets/behaviors-f1bee898f94fff2126806c33b72d3b50.css" />\n    \n    \n    \n    <link crossorigin="anonymous" media="all" integrity="sha512-ro9

In [ ]:
with open('webpage.html','w') as f:
    f.write(page_content)

## Beautiful soup
1. structure the page
2. extract info
3. create func for keeping info in dict/list
4. Use REST API if necessery. 

In [ ]:
!pip install beautifulsoup4 

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
page_doc = BeautifulSoup(page_content, 'html.parser')

In [ ]:
import pandas as pd

## CSV file(s)

1. save info in CSV
2. verify the csv file with pandas

## Get Info of each topic

In [ ]:
import os
def get_topic_page(topic_url):
    resp = requests.get(topic_url)
    if resp.status_code!=200:
        raise Exception(f'Failed to load{topic_url}')
    html_doc =  BeautifulSoup(resp.text,'html.parser')
    return html_doc

def get_repo_info(h1_tag,star_tag ):
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name =  a_tags[1].text.strip()
    repo_url = 'https://github.com'+a_tags[1]['href']
    stars = star_tag.text.strip()
    return username,repo_name,stars,repo_url

def get_topic_repo(html_doc):
    topic_repo_tags = html_doc.find_all('h1',{'class': 'f3 color-text-secondary text-normal lh-condensed'})
    star_tags =  html_doc.find_all('a',{'class': 'social-count float-none'})

    topic_repo_dict= {'username':[], 'repo_title': [], 'stars':[], 'repo_url':[],}
    for i in range(len(topic_repo_tags)):
        repo_info = get_repo_info(topic_repo_tags[i],star_tags[i])
        topic_repo_dict['username'].append(repo_info[0])
        topic_repo_dict['repo_title'].append(repo_info[1])
        topic_repo_dict['stars'].append(repo_info[2])
        topic_repo_dict['repo_url'].append(repo_info[3])
    return pd.DataFrame(topic_repo_dict)

def scrape_each_topic(topic_url, path):
    
    if os.path.exists(path):
        print(f'{path} File is exist.Skipping...')
        return
    topic_df = get_topic_repo(get_topic_page(topic_url))
    topic_df.to_csv(path,index=None)

Write a func to:
1. get the list from topics page
2. get the top repo list from individual topic page
3. create CSV of the top repo for each topic 

In [ ]:
def get_topic_titles(doc):
    topic_title_tag = doc.find_all('p',{'class':'f3 lh-condensed mb-0 mt-1 Link--primary'})
    topic_titles = []
    for title in topic_title_tag:
        topic_titles.append(title.text)
    return topic_titles

def get_topic_desc(doc):
    topic_desc_tag = doc.find_all('p',{'class':'f5 color-text-secondary mb-0 mt-1'})
    topic_desc = []
    for desc in topic_desc_tag:
        topic_desc.append(desc.text.strip())
    return topic_desc

def get_topic_link(doc):
    topic_link_tag = doc.find_all('a',{'class': 'd-flex no-underline'})
    topic_url = []
    for url in topic_link_tag:
        topic_url.append("https://github.com"+url['href'])
    return topic_url
    
def scrape_topics():
    topics_url = 'https://github.com/topics'
    resp = requests.get(topics_url)
    if resp.status_code!=200:
        raise Exception(f'Failed to load{topics_url}')
    html_doc =  BeautifulSoup(resp.text,'html.parser')
    
    topics_dict = {
        'title' : get_topic_titles(html_doc),
        'desc' : get_topic_desc(html_doc),
        'url' : get_topic_link(html_doc),
    }
    
    return pd.DataFrame(topics_dict)
# def scrape_topics_repo():

In [ ]:
def scrape_topics_repo():
    topics_df = scrape_topics()

    os.makedirs('data',exist_ok=True)

    for i,row in topics_df.iterrows():
        print('name: {}'.format(row["title"]))
        scrape_each_topic(row['url'], 'data/{}.csv'.format(row['title']))
    

In [ ]:
topics_df = scrape_topics()

In [ ]:
topics_df

download folder from colab

In [ ]:
!zip -r /content/data.zip /content/data

In [ ]:
 from google.colab import files
 files.download('/content/data.zip')